In [1]:
import pandas as pd
import numpy as np
import localsolver
import sys
import math

In [2]:
C_C_df = pd.read_csv(r'../Data/C_C.csv')
B_C_df = pd.read_csv(r'../Data/B_C.csv')

In [61]:
C_C_matrix = np.ones((174, 174)) * 1000000000
B_C_matrix = np.ones((141, 174)) * 1000000000

In [62]:
# c_c_matrix
OriginID = C_C_df['OriginID'].values.tolist()
DestinID = C_C_df['Destinatio'].values.tolist()
Length = C_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    C_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

In [63]:
# b_c_matrix
OriginID = B_C_df['OriginID'].values.tolist()
DestinID = B_C_df['Destinatio'].values.tolist()
Length = B_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    B_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

In [64]:
B_C_matrix

array([[1.34594182e+03, 1.12832324e+03, 9.20205159e+02, ...,
        2.58342697e+03, 2.13837845e+03, 2.41046658e+03],
       [1.47889756e+03, 1.26127897e+03, 1.05316089e+03, ...,
        3.16671011e+03, 2.84117958e+03, 3.11326771e+03],
       [1.00000000e+09, 1.00000000e+09, 1.00000000e+09, ...,
        1.00000000e+09, 1.00000000e+09, 1.00000000e+09],
       ...,
       [2.46791175e+03, 2.68553033e+03, 2.75565372e+03, ...,
        5.97591106e+02, 1.08299498e+03, 8.10906843e+02],
       [2.48596105e+03, 2.70357963e+03, 2.77370301e+03, ...,
        6.15640405e+02, 1.10104427e+03, 8.28956141e+02],
       [2.53038232e+03, 2.74800091e+03, 2.81812429e+03, ...,
        6.60061680e+02, 1.14546555e+03, 8.73377417e+02]])

In [65]:
nb_customers = 174
truck_capacity = 5
dist_matrix_data = C_C_matrix.tolist()
dist_depot_data = B_C_matrix.tolist()[20]
demands_data = np.random.randint(0, 2, nb_customers)
nb_trucks = 40
str_time_limit = "50"

In [66]:
with localsolver.LocalSolver() as ls:
    model = ls.model
    customers_sequences = [model.list(nb_customers) for _ in range(nb_trucks)]
    
    quantity = [None] * nb_trucks
    for k in range(nb_trucks):
        quantity[k] = [model.float(0, demands_data[i]) for i in range(nb_customers)]
        
    model.constraint(model.cover(customers_sequences))
    
    dist_matrix = model.array(dist_matrix_data)
    dist_depot = model.array(dist_depot_data)
    
    route_distances = [None] * nb_trucks
    trucks_used = [None] * nb_trucks
    
    for k in range(nb_trucks):
        sequence = customers_sequences[k]
        c = model.count(sequence)
        
        trucks_used[k] = model.count(sequence) > 0
        
        quantity_array = model.array(quantity[k])
        quantity_lambda = model.lambda_function(lambda j: quantity_array[j])
        route_quantity = model.sum(sequence, quantity_lambda)
        model.constraint(route_quantity <= truck_capacity)
        
        dist_lambda = model.lambda_function(lambda i: model.at(dist_matrix, sequence[i - 1], sequence[i]))
        route_distances[k] = model.sum(model.range(1, c), dist_lambda) + model.iif(
                    trucks_used[k],
                    dist_depot[sequence[0]] + dist_depot[sequence[c - 1]], 0)
        
    for i in range(nb_customers):
        # Each customer must receive at least its demand
        quantity_served = model.sum(quantity[k][i] * model.contains(customers_sequences[k], i)
            for k in range(nb_trucks))
        model.constraint(quantity_served >= demands_data[i])
            
    total_distance = model.sum(route_distances)
    
    model.minimize(total_distance)
#     model.close()
    model.close()

    ls.param.time_limit = int(str_time_limit)
    ls.solve()
    r_total_distance = total_distance.value
    customer_list = []
    for k in range(nb_trucks):
        c_list = []
        if trucks_used[k].value != 1:
            customer_list.append([])
            continue
        for customer in customers_sequences[k].value:
            c_list.append(customer)
        customer_list.append(c_list)

Push initial solution 100%
Model:  expressions = 37329, decisions = 7000, constraints = 215, objectives = 1
Param:  time limit = 50 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 94)
[  1 sec,    1740 itr]:   2.0002e+09
[  2 sec,   17584 itr]:  2.00007e+09
[  3 sec,   40000 itr]:  2.00006e+09
[  4 sec,   58616 itr]:  2.00006e+09
[  5 sec,   80000 itr]:  2.00006e+09
[  6 sec,   80000 itr]:  2.00006e+09
[  7 sec,  114723 itr]:  2.00006e+09
[  8 sec,  120000 itr]:  2.00006e+09
[  9 sec,  139334 itr]:  2.00006e+09
[ 10 sec,  160000 itr]:  2.00006e+09
[ optimality gap     ]:      100.00%
[ 11 sec,  160000 itr]:  2.00006e+09
[ 12 sec,  190085 itr]:  2.00006e+09
[ 13 sec,  200000 itr]:  2.00006e+09
[ 14 sec,  200000 itr]:  2.00006e+09
[ 15 sec,  210600 itr]:  2.00006e+09
[ 16 sec,  240000 itr]:  2.00006e+09
[ 17 sec,  240000 itr]:  2.00006e+09
[ 18 sec,  280000 itr]:  2.00006e+09
[ 19 sec,  296256 itr]:  2.00006e+09


In [67]:
print(r_total_distance)

2000056875.2011075


In [41]:
customer_list

[[],
 [],
 [33, 12, 6, 2, 1, 3, 0, 4, 5, 13, 35],
 [102, 97, 99, 98, 93, 91, 71, 64, 62, 70, 76],
 [],
 [],
 [123, 134, 135, 168, 169, 172, 173, 170, 158, 153, 142, 116],
 [],
 [67, 72, 79, 54, 56],
 [57, 92, 117, 130, 139, 147, 144, 149, 145, 148, 160, 17],
 [],
 [],
 [96,
  88,
  94,
  89,
  95,
  105,
  122,
  140,
  141,
  165,
  137,
  128,
  126,
  124,
  119,
  118,
  112,
  120,
  121,
  116],
 [103, 107, 108, 101, 100, 104, 111],
 [127, 152, 162, 167, 166, 161, 157, 156, 146, 138, 129],
 [],
 [18, 11, 8, 7, 16],
 [65, 74, 77, 73, 83, 82],
 [57, 86, 80, 75, 59, 58],
 [51, 43, 32, 45, 34, 42, 41, 39, 40, 38, 57],
 [],
 [],
 [57, 50, 44, 28, 27, 26, 30, 25, 21, 47, 37, 68, 69],
 [],
 [],
 [53, 31, 36, 22, 23, 19, 24, 29, 15, 20, 10, 9, 14, 57],
 [115, 132, 133, 136, 150, 154, 155, 159, 164, 171, 163],
 [],
 [96, 110, 109, 113, 114, 106, 90, 82],
 [],
 [],
 [44, 60, 78, 84, 81, 66, 63, 61, 55, 53, 57],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [143, 151, 131, 125, 87, 85, 48, 49, 52, 46

In [23]:
print(r_total_distance)

20076814.55788112


In [24]:
customer_list

[[],
 [],
 [],
 [],
 [7, 55, 61, 63, 66, 81, 84, 78, 60, 52, 49],
 [21, 25, 30, 26, 37],
 [57, 67, 72, 79, 54, 56, 50, 44],
 [],
 [114, 106, 102, 97, 99, 107, 123],
 [33, 14, 8, 20, 3, 0, 1, 47],
 [],
 [6, 12, 27, 35, 28, 13, 5, 4, 2],
 [109, 137, 140, 135],
 [110, 88, 94, 89, 95, 86, 59, 53, 51, 43, 32, 45],
 [],
 [],
 [],
 [],
 [105,
  103,
  116,
  142,
  154,
  155,
  159,
  160,
  148,
  145,
  149,
  144,
  147,
  164,
  171,
  163,
  152,
  127],
 [],
 [9, 85, 87, 125, 139, 131, 130, 17],
 [113, 122, 134, 129, 138, 146, 156, 157, 161, 167, 162, 166, 169, 165],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [80, 70, 62, 64, 71, 65, 74, 77, 73, 83, 91, 93, 92, 96, 68],
 [],
 [69, 34, 38, 41, 42, 39, 40],
 [],
 [16, 31, 46, 48, 36, 22, 23, 19, 24, 29, 18, 15, 11, 10],
 [58, 75, 76, 82, 90],
 [],
 [141,
  128,
  126,
  124,
  119,
  118,
  112,
  120,
  121,
  117,
  115,
  111,
  104,
  100,
  101,
  108,
  98,
  96],
 [],
 [132, 133, 136, 150, 151, 143, 153, 158, 170, 173, 172, 168, 113],
 [

In [ ]:
nb_customers = 174
truck_capacity = 5
dist_matrix_data = C_C_matrix.tolist()
dist_depot_data = B_C_matrix.tolist()[20]
demands_data = np.random.randint(0, 2, nb_customers)
nb_trucks = 40
str_time_limit = "50"

In [42]:
dist_depot_data

[1300.246823,
 1082.628237,
 874.5101578,
 1318.158443,
 922.3651787,
 963.7276419,
 870.1246349,
 1045.635534,
 1094.671976,
 909.2017114,
 1196.776467,
 1141.239976,
 648.340294,
 645.0517512,
 741.921893,
 1187.669634,
 853.4033343,
 10000000.0,
 1211.04004,
 1405.664773,
 1262.76636,
 1089.495207,
 1462.740495,
 1406.060137,
 1300.424857,
 798.7429946,
 601.4288034,
 470.857249,
 566.5050269,
 1265.520957,
 706.5121583,
 1276.916631,
 473.9920765,
 382.8078362,
 508.1920004,
 520.6685225,
 1551.428774,
 1405.992641,
 789.8610443,
 938.3642376,
 938.3642376,
 933.0038702,
 933.0038702,
 390.7546435,
 296.7330727,
 345.8450358,
 1417.785646,
 1472.470585,
 1563.06877,
 1652.513195,
 198.1958758,
 245.1326614,
 1798.016979,
 120.3290476,
 46.923712,
 1694.364113,
 16.0086498,
 46.50024105,
 124.4295311,
 353.5631474,
 1719.939694,
 1774.505838,
 549.8603279,
 1767.580794,
 545.3741273,
 624.804996,
 1838.243159,
 44.06487522,
 1251.76477,
 690.0624098,
 449.3068894,
 597.2453804,
 101

In [60]:
for c_list in customer_list:
    distance = 0
    if len(c_list) != 0:
        for i in range(len(c_list) - 1):

            fromC = c_list[i] - 1
            toC = c_list[i+1] - 1

            if i == 0:
                distance += dist_depot_data[fromC]
            else:
                distance += dist_matrix_data[fromC][toC]
        distance += dist_depot_data[c_list[-1] - 1]
    
    print(c_list)
    print(distance)
    print("++++++++++++++++++++++++++++++++++++++++++")

[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[33, 12, 6, 2, 1, 3, 0, 4, 5, 13, 35]
10913.6420748
++++++++++++++++++++++++++++++++++++++++++
[102, 97, 99, 98, 93, 91, 71, 64, 62, 70, 76]
6858.511611329999
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[123, 134, 135, 168, 169, 172, 173, 170, 158, 153, 142, 116]
7535.460405899999
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[67, 72, 79, 54, 56]
9077.018055
++++++++++++++++++++++++++++++++++++++++++
[57, 92, 117, 130, 139, 147, 144, 149, 145, 148, 160, 17]
6128.1055095
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[96, 88, 94, 89, 95, 105, 122, 140, 141, 165, 137, 128, 126, 124, 119, 118, 112, 120, 121, 116]
15879.97022559
+++++++++++++++++++++++++++++++++++++++

In [57]:
len(dist_depot_data)

174

In [48]:
distance

10405.4500744

In [43]:
customer_list

[[],
 [],
 [33, 12, 6, 2, 1, 3, 0, 4, 5, 13, 35],
 [102, 97, 99, 98, 93, 91, 71, 64, 62, 70, 76],
 [],
 [],
 [123, 134, 135, 168, 169, 172, 173, 170, 158, 153, 142, 116],
 [],
 [67, 72, 79, 54, 56],
 [57, 92, 117, 130, 139, 147, 144, 149, 145, 148, 160, 17],
 [],
 [],
 [96,
  88,
  94,
  89,
  95,
  105,
  122,
  140,
  141,
  165,
  137,
  128,
  126,
  124,
  119,
  118,
  112,
  120,
  121,
  116],
 [103, 107, 108, 101, 100, 104, 111],
 [127, 152, 162, 167, 166, 161, 157, 156, 146, 138, 129],
 [],
 [18, 11, 8, 7, 16],
 [65, 74, 77, 73, 83, 82],
 [57, 86, 80, 75, 59, 58],
 [51, 43, 32, 45, 34, 42, 41, 39, 40, 38, 57],
 [],
 [],
 [57, 50, 44, 28, 27, 26, 30, 25, 21, 47, 37, 68, 69],
 [],
 [],
 [53, 31, 36, 22, 23, 19, 24, 29, 15, 20, 10, 9, 14, 57],
 [115, 132, 133, 136, 150, 154, 155, 159, 164, 171, 163],
 [],
 [96, 110, 109, 113, 114, 106, 90, 82],
 [],
 [],
 [44, 60, 78, 84, 81, 66, 63, 61, 55, 53, 57],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [143, 151, 131, 125, 87, 85, 48, 49, 52, 46